In [ ]:
!pip install autots python-dotenv huggingface_hub


In [1]:
import pandas as pd

# 파일 경로
train_file_path = 'data/train.csv'
test_file_path = 'data/test.csv'

# 데이터 불러오기
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# train_data, test_data에 날짜 열 추가
train_data['Date'] = pd.date_range(start='2020-01-01', periods=len(train_data), freq='D')
test_data['Date'] = pd.date_range(start='2020-04-10', periods=len(test_data), freq='D')

# 결과 확인
train_data.head(), test_data.head()

(            ID       x_0       x_1       x_2       x_3       x_4       x_5  \
 0  TRAIN_00000  1.006187 -1.962566  1.247535  0.926500 -0.265766 -1.789301   
 1  TRAIN_00001  1.024647 -2.472625  1.144386  0.846499 -0.287336 -1.756679   
 2  TRAIN_00002  1.062444 -2.451003  1.186546  0.873599 -0.257828 -1.802735   
 3  TRAIN_00003  1.089189 -2.458470  1.184531  0.810867 -0.276517 -1.787739   
 4  TRAIN_00004  1.023323 -2.133468  1.242266  0.939837 -0.264515 -1.792044   
 
         x_6       x_7       x_8       x_9      x_10          y       Date  
 0  0.470004 -0.139467  0.623996  0.320359  0.078612  83.424500 2020-01-01  
 1  0.503860 -0.219545  0.697607  0.238306  0.081778  79.374109 2020-01-02  
 2  0.498045 -0.194247  0.684134  0.259392  0.095003  82.181616 2020-01-03  
 3  0.503359 -0.201923  0.686394  0.245736  0.091737  83.006586 2020-01-04  
 4  0.470478 -0.142896  0.623778  0.314610  0.078987  83.051434 2020-01-05  ,
           ID       x_0       x_1       x_2       x_3       x

In [2]:
from autots import AutoTS

# AutoTS 모델 초기화
model = AutoTS(
    forecast_length=10,         # 예측하고자 하는 기간
    frequency='D',              # 데이터가 일별 주기이므로 'D'로 설정
    ensemble='simple',          # 앙상블 전략
    max_generations=5,          # 진화 과정에서 반복할 세대 수
    n_jobs='auto'               # 병렬 처리
)

Using 6 cpus for n_jobs.


In [ ]:
# 모델 학습 (train_data에서 'y' 값을 예측하도록 학습)
model = model.fit(train_data, date_col='Date', value_col='y')

In [ ]:
# test 데이터로 예측 수행
test_predictions = model.predict(test_data)
model.export_template()
# 예측 결과에서 포인트 예측값 가져오기
test_forecast = test_predictions.forecast

# 예측값 출력
print(test_forecast)

In [ ]:
# sample_submission.csv 파일 불러오기
submission_file_path = 'data/sample_submission.csv'
submission_data = pd.read_csv(submission_file_path)

# 예측 결과를 sample_submission 형식에 맞춰 'y' 컬럼에 삽입
submission_data['y'] = test_forecast.values

# 결과를 CSV로 저장 (경로는 '/mnt/data/submission_result.csv')
output_path = 'sub.csv'
submission_data.to_csv(output_path, index=False)

output_path  # 출력 경로 확인

In [4]:
model.export_best_model('a.csv')